# Introduction

In [ ]:
# retrieve the sp code
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the road network and demand inputs
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/north_berkeley_edges_clean.csv" -O traffic_inputs/berkeley_edges.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/north_berkeley_nodes_clean.csv" -O traffic_inputs/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_10pn.csv" -O traffic_inputs/od_10pn.csv

In [ ]:
import pandas as pd
from sp import interface

In [ ]:
# read network files
edges_df = pd.read_csv('traffic_inputs/berkeley_edges.csv')
display(edges_df.head(2))
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
display(nodes_df.head(2))

# read travel demand (origin-destination) files
od_df = pd.read_csv('traffic_inputs/od_10pn.csv')
display(od_df.head(2))

In [ ]:
### make road network graph
g = interface.from_dataframe(edges_df, 'start_node_id', 'end_node_id', 'fft')

In [ ]:
# get path
def get_path(origin, destin):
    sp = g.dijkstra(origin, destin)
    sp_dist = sp.distance(destin)

    if sp_dist > 10e7:
        route = []
    else:
        route = [(start_node_id, end_node_id) for (start_node_id, end_node_id) in sp.route(destin)]
    sp.clear()
    
    return route, sp_dist

### example of path calculation
origin = 91      ### the origin node id of a trip
destin = 237     ### the end node id of a trip
route, distance = get_path(origin, destin) ### get the shortest path between the origin and destination of the trip
print('The shortest path between {} and {} is {}'.format(origin, destin, route))
print('The path/route object is a {}, the first elment is {}'.format(type(route), route[0]))
print('The trip travel time is {:.2f} minutes.'.format(distance/60))

### example of traffic volume calculation
traffic_volume_dictionary = dict()
for path_link in route:
  try:
    traffic_volume_dictionary[path_link] += 1
  except KeyError:
    traffic_volume_dictionary[path_link] = 1
print( 'The link that connects node 151 and 226 has {} trip(s) passing through it (so far)'.format( traffic_volume_dictionary[ (151, 226) ] ) )
print( 'The link_id of the link that connects node 151 and 226 is {}'.format( edges_df.loc[(edges_df['start_node_id']==151) & (edges_df['end_node_id']==226), 'link_id'].values[0] ) )


Given the inputs (`edges_df`, `nodes_df`, `od_df`, `g`, and the `get_path` fuction),
- Task 1: calculate the path for each od pair
- Task 2: compute the number of trips that pass each link
- Task 3: save the result of Task 2 in a CSV file `quiz3_[your_name].csv`. The file should look like:

| link_id | volume      |
|---------|-------------|
| 0       | ...         |
| 1       | ...         |
| ...     | ...         |

Hints:
- Loop through the `od` dictionary to compute the path for each od pair;
- The path/`route` of each od pair is a list. Each list element denotes a road link by its start and end node id. For example, in the example above, the path between node 354 and node 196 is `[(354, 124), (124, 69), (69, 267), ...]`. Its first element `(354, 124)` denotes the road link that connects node 354 and node 196.
- You can use tuple (e.g, `(354, 124)`) as the key in Python dictionary. Whenever a new link is being used (e.g., whenever a new path is computed and `(354, 124)` is in the path), update the corresponding dictionary value by one to count its total usage.
- When you are finding the `link_id` of each link based on its start and end node id, beware that links are directed, meaning that link `(354, 124)` is different from link `(124, 354)`.
- There are lots of other ways to accomplish the above tasks, e.g., using Pandas groupby and merge functions. Please choose a method that works for you.

In [ ]:
### YOUR CODE HERE ###
### create a dataframe that counts the traffic volume on each road link under the given travel demand
### name the output file as "quiz3_[your_name].csv"


traffic_volume_df.to_csv('quiz3_[].csv'.format(your_name), index=False)